# Traveling Salesman Problem via Genetic algorithms

Francesco Andreuzzi -- Data Science and Scientific Computing

Final project for the course "Computational Physics Laboratory" (2022/2023)

## The problem

The Traveling Salesman Problem (TSP) is a well known problem in Computer Science. It consists in finding the optimal (i.e. cheapest) tour which visits all cities in the instance, given a matrix of costs to travel from one to another. 
$$
\normalsize
\begin{align*}
\text{Minimize}& \qquad \sum_{i=1}^N \sum_{j=1}^N x_{ij} d_{ij} \\
\text{with}& \qquad \sum_{j=1}^N x_{ij} = 1 \qquad i = 1, \dots N \quad (\text{Exactly one incoming})\\
& \qquad \sum_{i=1}^N x_{ij} = 1 \qquad j = 1 \dots N \quad (\text{Exactly one outgoing})
\end{align*}
$$

It's NP-hard, therefore exact algorithms are unfeasible for non-trivial instances.

## Genetic algorithms overview

Genetic algorithms (GA) are approximation randomized algorithms which iteratively optimize a partial set of viable solutions to a given problem, often called *chromosomes*. The optimization is carried out on a given *fitness* function, which in our case is the cost of a given tour.

Optimization happens by a peculiar process called *crossover*, which consists in merging two parent solutions into a new one, hopefully retained the best parts of the ancestors. Many *crossover operators* exist for the TSP problem, some of them are fully deterministic (e.g. `CX`, `CX2`), while others are based on a stochastic component (e.g. `OX`, `PMX`).

In addition to crossover, to further mimic natural biological evolution, GA leverage *mutation* to slightly modify a subset of exising chromosomes (choosed randomly). This often provides solutions not easily achievable by means of crossover, and is critical in fully exploring the solution manifold. Due to their intrinsic stochastic definition, mutation operator are always randomized procedures which operate on a single chromosome.

## Structure of the project

We briefly describe the structure of the GitHub repository.

### Languages

The problem was tackled using Python and Fortran 90, leveraging [f2py](https://numpy.org/doc/stable/f2py/index.html) to link the two languages. Python was used to develop the high level driver for the genetic framework, the problem loading and the data collection layer, and to perform final data analysis, due to its strong versatility. Fortran was used to develop the low-level mutation and crossover operators.

Thanks to the link between Python and Fortran it has been possible to test the fortran code by using the high level Python testing framework [pytest](https://pytest.org), which among standard testing traits provides many comfortable features like a rich CLI and [fixtures](https://docs.pytest.org/en/6.2.x/fixture.html).

### Directories

```diff
$ tree -I 'venv|__pycache__|*.mod' --dirsfirst
.
├── data
-   │   # Problem data in XML format
│   │   ...
├── data-loader
-   │   # Responsible for loading problem data from XML
│   ├── load.py
-   │   # Problem representation as a Python class
│   └── problem.py
├── run
-   │   # Argparse CLI + utilities (logging, text display)
│   ├── data_storage.py
│   ├── printer.py
│   ├── run.py
│   └── utils.py
├── tests
-   │   # Pytest testing for low-level Fortran routines
│   ├── crossover
│   │   ├── test_crossover.py
│   │   ├── test_cx2-original.py
│   │   ├── test_cx2.py
│   │   ├── test_cx.py
│   │   ├── test_ox.py
│   │   └── test_pmx.py
│   ├── mutation
│   │   ├── test_central_inverse.py
│   │   ├── test_mutation.py
│   │   ├── test_reverse_sequence.py
│   │   └── test_twors.py
│   ├── test_problem.py
│   └── test_utils.py
├── tsp-genetic-f90
-   │   # Fortran implementation of low-level operators
│   ├── crossover
│   │   ├── cx2.f90
│   │   ├── cx2-original.f90
│   │   ├── cx.f90
│   │   ├── ox.f90
│   │   └── pmx.f90
│   ├── mutation
│   │   ├── central_inverse.f90
│   │   ├── reverse_sequence.f90
│   │   └── twors.f90
│   ├── problem.f90
│   └── utils.f90
├── tsp-genetic-py
-   │   # Genetic framework implementation
│   ├── configuration.py
│   ├── crossover.py
│   ├── evolve.py
│   ├── idata_logger.py
│   └── mutation.py
├── LICENSE
├── Makefile
├── README.md
-   # We are here
├── tsp-genetic-algorithms.ipynb
-   # Fortran code compiled with f2py
└── tsp_genetic.cpython-39-x86_64-linux-gnu.so
```

In [1]:
import utils
from configuration import Configuration

# High level objects
We defined some high level objects to ease starting a genetic optimization run.

## Problem object
TSP problems are taken from the [TSPLIB](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/) repository, and are stored in the directory `data/` of the project. We represent them in Python with the [dataclass](https://docs.python.org/3/library/dataclasses.html) `Problem`, which contains the fields `name`, `cost_matrix` and `optimal_tour`.

In [3]:
from load import build_problem

build_problem("a280")

ModuleNotFoundError: No module named 'load'

## Configuration

We defined the dataclass `Configuration` to pack all needed configurations for a given genetic optimization run:

In [3]:
from configuration import Configuration
from dataclasses import fields

[field.name for field in fields(Configuration)]

['population_size',
 'elite_size',
 'n_generations',
 'mutation_operator',
 'mutation_probability',
 'mutation_function_degree',
 'crossover_operator',
 'crossover_strategy',
 'crossover_retainment',
 'next_generation_policy',
 'print_every']

# Genetic optimization

- Crossover operators
- Crossover strategy
- Mutation operators
- Mutation probability evolution
- Next generation policy & elitism

## Crossover operators

First of all we need to define configurations for all the run we intend to perform.

In [4]:
from dataclasses import replace
from configuration import (
    Crossover,
    CrossoverStrategy,
    CrossoverRetainment,
    NextGenerationPolicy,
)
from mutation import Mutation

base_config = Configuration(
    population_size=250,
    elite_size=0,
    n_generations=1000,
    crossover_operator=None,
    crossover_strategy=CrossoverStrategy.FITNESS_RANDOM_PAIRS,
    crossover_retainment=CrossoverRetainment.FIRST,
    next_generation_policy=NextGenerationPolicy.BEST,
    mutation_operator=Mutation.TWORS,
    mutation_probability=0.15,
    mutation_function_degree=0,
    print_every=100,
)
ox_config = replace(base_config, crossover_operator=Crossover.OX)
pmx_config = replace(base_config, crossover_operator=Crossover.PMX)
cx_config = replace(base_config, crossover_operator=Crossover.CX)
cx2_config = replace(base_config, crossover_operator=Crossover.CX2)

In [ ]:
from utils import run_optimizations

N = 10
problem = build_problem("a280")
ox_best_data, ox_mean_data = run_optimizations(N, problem, ox_config)
pmx_best_data, pmx_mean_data = run_optimizations(N, problem, pmx_config)
cx_best_data, cx_mean_data = run_optimizations(N, problem, cx_config)
cx2_best_data, cx2_mean_data = run_optimizations(N, problem, cx2_config)

In [ ]:
import numpy as np
from utils import plot_data
import matplotlib.pyplot as plt

generations = base_config.print_every * np.arange(
    1, base_config.n_generations // base_config.print_every + 1
)

plt.figure(figsize=(10, 6))
plot_data(
    generations,
    ("OX", "PMX", "CX", "CX2"),
    (ox_best_data, pmx_best_data, cx_best_data, cx2_best_data),
    n_runs=N
)

It's quite evident that operators `OX` and `PMX` achieve much better results. Indeed, such operators try to preserve the relative order in the path, as opposed to cyclic operators which often disrupt the sequence. This finding is widely accepted in literature ([2]).

## Mutation evolution

As noted in [6], it's desirable to increase the mutation probability as the generation index increases, since the population of chromosomes tends to stagnate towards repeated high-fitness individuals. For this reason, extending the work presented in [6], we implemented a flexible evolution policy for the mutation probability, i.e. the probability that a given chromosome is subject to a mutation after crossover happens. The policy is parametrized by an integer number $k$, which acts as the polynomial degree of the function which describes the evolution of the probability in time.

The mutation probability $m$ for the generation $g$ is computed as follows:
$$
\normalsize
m(g) = \frac{M}{G^k} g^k
$$

given the total number of generations $G$ and the target mutation probability $M$.

In [ ]:
import matplotlib.pyplot as plt
from utils import setup_matplotlib

setup_matplotlib()

g = np.arange(501)
M = 0.2


def plot_evolution(k):
    plt.plot(g, M / g[-1] ** k * g**k, label=f"k={k}")


for i in range(6):
    plot_evolution(i)

plt.legend()

plt.xlabel("Generation count")
plt.ylabel("m(g)")
plt.title(f"M = {M}, G = {g[-1]}")
plt.grid()

plt.show()

We now verify whether varying the mutation rate linearly, quadratically or cubically makes any difference in the best individual produced by the optimization. We're going to use again `OX` crossover as it was the best candidate of the previous experiment.

In [ ]:
ox1_config = replace(ox_config, mutation_function_degree=1)
ox2_config = replace(ox_config, mutation_function_degree=2)
ox3_config = replace(ox_config, mutation_function_degree=3)

ox1_best_data, ox1_best_data = run_optimizations(N, problem, ox1_config)
ox2_best_data, ox2_best_data = run_optimizations(N, problem, ox2_config)
ox3_best_data, ox3_best_data = run_optimizations(N, problem, ox3_config)

In [ ]:
plt.figure(figsize=(10, 6))
plot_data(
    generations,
    ("OX", "OX - 1", "OX - 2", "OX - 3"),
    (ox_best_data, ox1_best_data, ox2_best_data, ox3_best_data),
    should_plot_std=False,
    n_runs=N
);

From the plot above, it looks like the well-known observation that varying the mutation probability throughout the optimization run has been confirmed, but there's not much difference in how it varies (at least as long as polynomial functions are taken into account).

## Mutation operators
We now turn to the selection of an optimal mutation operator, among those which we implemented as part of the project:
- Twors (simple inversion)
- Central inverse
- Reverse sequence

In [ ]:
ox_twors_config = replace(ox_config, mutation_function_degree=1)
ox_ci_config = replace(ox_config, mutation_function_degree=1, mutation_operator=Mutation.CENTRAL_INVERSE)
ox_rs_config = replace(ox_config, mutation_function_degree=1, mutation_operator=Mutation.REVERSE_SEQUENCE)

ox_twors_best_data, ox1_best_data = run_optimizations(N, problem, ox_twors_config)
ox_ci_best_data, ox2_best_data = run_optimizations(N, problem, ox_ci_config)
ox_rs_best_data, ox3_best_data = run_optimizations(N, problem, ox_rs_config)

In [ ]:
plt.figure(figsize=(10, 6))
plot_data(
    generations,
    ("Twors", "Central inverse", "Reverse sequence"),
    (ox_twors_best_data, ox_ci_best_data, ox_rs_best_data),
    should_plot_std=False,
    n_runs=N
);

# Other topics

## Implementation of `CX2`

The MATLAB code in [3] proposed as the official implementation of `CX2` is clearly inefficient due to the frequent restort to the `find()` function, which is $\Theta(N)$. Since the crossover requires a linear scan of the array, the implementation in [3] is $\Theta(N^2)$.

For this reason we re-implemented it by leveraging a pre-built index, which makes the function $\Theta(N)$.

In [ ]:
from tsp_genetic import cx2, cx2_original
import perfplot

perfplot.show(
    setup=lambda n: ((np.random.permutation(n) + 1, np.random.permutation(n) + 1),),
    kernels=[
        lambda a: cx2_original(*a),
        lambda a: cx2(*a),
    ],
    labels=["Original", "With index"],
    n_range=[2**k for k in range(15)],
    show_progress=False,
)

setup_matplotlib()

# Conclusion

## Unexplored directions

It could be interesting to take into account various ways to generate the initial generation of chromosomes. In this project we've only considered the randomized permutation of the list of cities, but more refined approaches exist to guarantee a better coverage on the solution manifold.

## References

1. Deep, Kusum, and Hadush Mebrahtu. "New variations of order crossover for travelling salesman problem." International Journal of Combinatorial Optimization Problems and Informatics 2.1 (2011): 2-13.
1. Potvin, Jean-Yves. "Genetic algorithms for the traveling salesman problem." Annals of Operations Research 63 (1996): 337-370.
1. Hussain, Abid, et al. "Genetic algorithm for traveling salesman problem with modified cycle crossover operator." Computational intelligence and neuroscience 2017 (2017).
1. Kumar, Rakesh, Girdhar Gopal, and Rajesh Kumar. "Novel crossover operator for genetic algorithm for permutation problems." International Journal of Soft Computing and Engineering (IJSCE) 3.2 (2013): 252-258.
1. Abdoun, Otman, Jaafar Abouchabaka, and Chakir Tajani. "Analyzing the performance of mutation operators to solve the travelling salesman problem." arXiv preprint arXiv:1203.3099 (2012).
1. Hassanat, Ahmad, et al. "Choosing mutation and crossover ratios for genetic algorithms—a review with a new dynamic approach." Information 10.12 (2019): 390.
1. f2py official documentation: https://numpy.org/doc/stable/f2py/

## Open source software

- argparse: https://docs.python.org/3/library/argparse.html
- f2py: https://numpy.org/doc/stable/f2py/
- Matplotlib: https://matplotlib.org
- NumPy: https://numpy.org
- Python standard library: https://docs.python.org/3/library/
- pytest: https://pytest.org
- perfplot: https://github.com/nschloe/perfplot